In [5]:
#!/usr/bin/env python                                                                                                                                                                                        

# Import libraries                                                                                                                                                                                           
import sys
import argparse
import time

class args():
    pass

args = args
args.n_splits=3
args.model_name = "svm"
args.y_prefix = "INCLUDE"
args.y_prefix = "6 -"


t0 = time.time()

# Establish what task number we have if running from slurm, otherwise just get a random number                                                                                                               
try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    test = False
except:
    import random
    rank = random.randint(0,args.n_splits**2)
    print(rank)
    test = True

rank_i = rank//args.n_splits
rank_j = rank%args.n_splits

# Import the rest of the libraries                                                                                                                                                                           
import gc
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold
import pickle
import re
import sklearn

# Load data                                                                                                                                                                                                  
df = pd.read_csv('../data/0_labelled_documents.csv')

df = (df
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)



0


In [6]:
print(sys.version)

print(sklearn.__version__)

3.6.9 (default, Mar 15 2022, 13:55:28) 
[GCC 8.4.0]
0.24.2


In [7]:

args.y_prefix = "8 "
if test:
    print("TESTING WITH 100 documents")
    #df = df.head(150)


if len(args.y_prefix) < 2:
    args.y_prefix+=" "
cols = [x for x in df.columns if re.match(f"^{args.y_prefix}",x)]
print(cols)
num_labels=len(cols)


# If the target is inclusion, use only those documents for which we have a non-na value                                                                                                                      
# Otherwise, only use those documents which are included                                                                                                                                                     
# also define what subset is to be treated as a random representative sample                                                                                                                                 
# For labels beyond inclusion, we treat all those that are representative of the included                                                                                                                    
# studies as representative                                                                                                                                                                                  

if "INCLUDE" in args.y_prefix:
    df = df.loc[pd.notna(df["INCLUDE"]),:].reset_index(drop=True)
    df['random'] = df['representative_sample']
else:
    df = df[df['INCLUDE']==1]
    df['random'] = df['representative_relevant']




TESTING WITH 100 documents
['8 - 01. AFOLU', '8 - 02. Buildings', '8 - 03. Industry', '8 - 04. Energy', '8 - 05. Transport', '8 - 06. Waste', '8 - 15. Cross-sectoral', '8 - 08. Freshwater resources']


In [8]:
if "INCLUDE" in args.y_prefix:
    df = df.loc[pd.notna(df["INCLUDE"]),:].reset_index(drop=True)
    df['random'] = df['representative_sample']
else:
    df = df[df['INCLUDE']==1]
    df['random'] = df['representative_relevant']


# Turn the columns into target variables and get class-weights to counteract class imbalances                                                                           
if len(cols)==1:
    scorer = "f1"
    y_var = cols[0]
    df = df.loc[pd.notna(df[y_var]),:].reset_index(drop=True)
    print(df.shape)
    df['labels'] = list(df[y_var].values.astype(int))
    y = df['labels']
    cw = df[(df['random']==1) & (df[y_var]==0)].shape[0] / df[(df['random']==1) & (df[y_var]==1)].shape[0]
    class_weight={0:1, 1:cw}
    scorer = "F1"
else:
    scorer = "f1_macro"
    num_labels = len(cols)
    df = df.replace(2,1)
    df['labels'] = list(df[cols].values.astype(int))
    df = df.dropna(subset=cols)
    df = df.reset_index(drop=True)
    y = np.matrix(df[cols])

    scorer = "f1_macro"
    #class_weight = []
    class_weight = {}
    for i, t in enumerate(cols):
        print(t)
        cw = df[(df['random']==1) & (df[t]==0)].shape[0] / df[(df['random']==1) & (df[t]==1)].shape[0]
        #class_weight.append({0:1, 1:cw}) 
        class_weight[i] = cw




import cv_setup as cvs

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

if len(cols)>1:
    pipeline = cvs.mpipeline
    parameters = cvs.mparameters
    for p in parameters:
        p['clf__estimator__class_weight'] = ['balanced',None]
else:
    pipeline = cvs.pipeline
    parameters = cvs.parameters
    for p in parameters:
        p['clf__class_weight'] = ['balanced',class_weight]

# for p in parameters:
#     p['clf__estimator__class_weight'] = ['balanced',class_weight]

nonrandom_index = df[df['random']!=1].index




8 - 01. AFOLU
8 - 02. Buildings
8 - 03. Industry
8 - 04. Energy
8 - 05. Transport
8 - 06. Waste
8 - 15. Cross-sectoral
8 - 08. Freshwater resources


ZeroDivisionError: division by zero

In [5]:
for p in parameters:
    for key, value in p.items():
        if len(value) > 1:
            p[key] = (value[1],)
        #p[key] = value[0]
        pass
        
parameters

[{'vect__max_df': (0.75,),
  'vect__min_df': (10,),
  'vect__ngram_range': ((1, 2),),
  'clf__estimator__kernel': ['rbf'],
  'clf__estimator__gamma': (0.0001,),
  'clf__estimator__C': (100.0,),
  'clf__estimator__class_weight': (None,)},
 {'vect__max_df': (75,),
  'vect__min_df': (10,),
  'vect__ngram_range': ((1, 2),),
  'clf__estimator__kernel': ['linear'],
  'clf__estimator__C': (100.0,),
  'clf__estimator__class_weight': (None,)}]

In [6]:
args.resume = "False"

outer_cv = cvs.KFoldRandom(args.n_splits, df.index, nonrandom_index, discard=False)

for k, (train, test) in enumerate(outer_cv):
    print(k)
    if k!=rank_j:
        continue
    fname = f'cv/df_{df.shape[0]}_cv_results_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}.csv'
    if args.resume == "True":
        inner_scores = pd.read_csv(fname)
        best_model = inner_scores.sort_values('rank_test_score').to_dict('records')[0]
        if best_model['param_clf__class_weight'] != "balanced":
            best_model['param_clf__class_weight'] = ast.literal_eval(best_model['param_clf__class_weight'])
        clf = Pipeline([
            ('vect', TfidfVectorizer(
                max_df=best_model['param_vect__max_df'],
                min_df=best_model['param_vect__min_df'],
                ngram_range=ast.literal_eval(best_model['param_vect__ngram_range'])
            )),
            ('clf', SVC(
                probability=True,
                C=best_model['param_clf__C'],
                class_weight=best_model['param_clf__class_weight'],
                kernel=best_model['param_clf__kernel'],
                gamma=best_model['param_clf__gamma']
            )),
        ])
        clf.fit(df.loc[train,'content'],y[train,:])
    else:
        inner_cv = cvs.KFoldRandom(args.n_splits, train, nonrandom_index, discard=False)
        for a,b in inner_cv:
            
            #print(y[a,:].sum(axis=0))
            #print(y[b,:].sum(axis=0))
        inner_cv = cvs.KFoldRandom(args.n_splits, train, nonrandom_index, discard=False)
        #clf = GridSearchCV(pipeline, parameters, scoring="f1", n_jobs=8, verbose=1, cv=inner_cv)                                                                       
        clf = GridSearchCV(pipeline, parameters, scoring=scorer, n_jobs=8, verbose=1, cv=inner_cv)
        print(df.loc[train,'labels'])
        clf.fit(df.loc[train, 'content'].values, y[train,:])




IndentationError: expected an indented block (<ipython-input-6-e124ec8baa94>, line 36)

In [ ]:
set(b) & set(test)

In [ ]:
y_preds = clf.predict_proba(df.loc[test,'content'])
eps = cvs.evaluate_preds(y[test], y_preds, cols)

In [11]:
eps

{'ROC AUC micro': 1.0,
 'F1 micro': 0.9430523917995445,
 'precision micro': 1.0,
 'recall micro': 0.8922413793103449,
 'ROC AUC macro': 1.0,
 'F1 macro': 0.8,
 'precision macro': 0.8,
 'recall macro': 0.8,
 'ROC AUC weighted': 1.0,
 'F1 weighted': 0.8922413793103449,
 'precision weighted': 0.8922413793103449,
 'recall weighted': 0.8922413793103449,
 'ROC AUC samples': nan,
 'F1 samples': 0.8977498691784406,
 'precision samples': 0.9230769230769231,
 'recall samples': 0.8841575091575092,
 'ROC AUC - 4 - 1. Economic instruments': 1.0,
 'precision - 4 - 1. Economic instruments': 1.0,
 'recall - 4 - 1. Economic instruments': 1.0,
 'F1 - 4 - 1. Economic instruments': 1.0,
 'accuracy - 4 - 1. Economic instruments': 1.0,
 'n_target - 4 - 1. Economic instruments': 111.0,
 'ROC AUC - 4 - 2. Regulatory Instruments': 1.0,
 'precision - 4 - 2. Regulatory Instruments': 0.0,
 'recall - 4 - 2. Regulatory Instruments': 0.0,
 'F1 - 4 - 2. Regulatory Instruments': 0.0,
 'accuracy - 4 - 2. Regulatory Ins

In [5]:

args.resume="True"
k = 0
fname = f'cv/df_{df.shape[0]}_cv_results_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}.csv'
if args.resume:
    inner_scores = pd.read_csv(fname)
    best_model = inner_scores.sort_values('rank_test_score').to_dict('records')[0]
    if best_model['param_clf__class_weight'] != "balanced":
        best_model['param_clf__class_weight'] = ast.literal_eval(best_model['param_clf__class_weight'])

best_model

FileNotFoundError: [Errno 2] No such file or directory: 'cv/df_588_cv_results_4 -_svm_0.csv'

In [ ]:
type(best_model)

In [6]:
outer_cv = cvs.KFoldRandom(args.n_splits, df.index, nonrandom_index, discard=False)

for k, (train, test) in enumerate(outer_cv):
    if k!=rank_j:                                                                                                                                                                                            
        continue                                                                                                                                                                                             
#     inner_cv = cvs.KFoldRandom(args.n_splits, train, nonrandom_index, discard=False)                                                                                                                         
#     clf = GridSearchCV(pipeline, parameters, scoring="f1", n_jobs=8, verbose=1, cv=inner_cv)                                                                                                                 
#     clf.fit(df.loc[train, 'content'], df.loc[train,'labels'])                                                                                                                                                

#     inner_scores = pd.DataFrame(clf.cv_results_)
#     fname = f'cv/df_{df.shape[0]}_cv_results_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}.csv'


#     inner_scores.to_csv(fname, index=False)
#     y_preds = clf.predict_proba(df.loc[test,'content'])#[:,1]                                                                                                                                                 
#     if len(cols)==1:
#         y_preds = y_preds[:,1]
#     eps = cvs.evaluate_preds(df.loc[test,'labels'], y_pred, cols)
#     best_params = inner_scores.sort_values('mean_test_score',ascending=False).to_dict('records')[0]['params']
#     for key, value in best_params.items():
#         eps[key] = value
#     eps["rank_k"] = rank_i


#     fname = f'cv/df_{df.shape[0]}_cv_results_outer_nonrandom_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}.csv'
#     pd.DataFrame.from_dict([eps]).to_csv(fname, index=False)
#     fname = f'cv/df_{df.shape[0]}_y_preds_nonrandom_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}'
#     np.save(fname,y_preds)
#     fname = f'cv/df_{df.shape[0]}_y_pred_nonrandom_ids_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}'
#     np.save(fname,df.loc[test,"id"])


In [12]:
args.resume = "True"
fname = f'cv/df_{df.shape[0]}_cv_results_{args.y_prefix}_{args.model_name.replace("/","__")}_{k}.csv'
if args.resume=="True":
	inner_scores = pd.read_csv(fname)


In [36]:
best_model = inner_scores.sort_values('rank_test_score').to_dict('records')[0]
if best_model['param_clf__class_weight'] != "balanced":
    best_model['param_clf__class_weight'] = ast.literal_eval(best_model['param_clf__class_weight'])
best_model

{'mean_fit_time': 9.53432289759318,
 'mean_score_time': 0.9220480918884276,
 'mean_test_score': 0.7097106654117254,
 'mean_train_score': 0.8124689253447457,
 'param_clf__C': 100.0,
 'param_clf__class_weight': 'balanced',
 'param_clf__gamma': 0.001,
 'param_clf__kernel': 'rbf',
 'param_vect__max_df': 0.75,
 'param_vect__min_df': 15,
 'param_vect__ngram_range': '(1, 1)',
 'params': "{'clf__C': 100.0, 'clf__class_weight': 'balanced', 'clf__gamma': 0.001, 'clf__kernel': 'rbf', 'vect__max_df': 0.75, 'vect__min_df': 15, 'vect__ngram_range': (1, 1)}",
 'rank_test_score': 3,
 'split0_test_score': 0.7246376811594203,
 'split0_train_score': 0.81875,
 'split1_test_score': 0.7447154471544716,
 'split1_train_score': 0.7984000000000001,
 'split2_test_score': 0.6597131681877445,
 'split2_train_score': 0.8202567760342369,
 'std_fit_time': 0.14415523327554391,
 'std_score_time': 0.016128296744403837,
 'std_test_score': 0.036269105441615386,
 'std_train_score': 0.009967232607277214}

In [15]:
clf.estimator.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__norm', 'vect__preprocessor', 'vect__smooth_idf', 'vect__stop_words', 'vect__strip_accents', 'vect__sublinear_tf', 'vect__token_pattern', 'vect__tokenizer', 'vect__use_idf', 'vect__vocabulary', 'clf__C', 'clf__break_ties', 'clf__cache_size', 'clf__class_weight', 'clf__coef0', 'clf__decision_function_shape', 'clf__degree', 'clf__gamma', 'clf__kernel', 'clf__max_iter', 'clf__probability', 'clf__random_state', 'clf__shrinking', 'clf__tol', 'clf__verbose'])

In [37]:
import ast
clf = Pipeline([
    ('vect', TfidfVectorizer(
        max_df=best_model['param_vect__max_df'],
        min_df=best_model['param_vect__min_df'],
        ngram_range=ast.literal_eval(best_model['param_vect__ngram_range'])
    )),
    ('clf', SVC(
        probability=True,
        C=best_model['param_clf__C'],
        class_weight=best_model['param_clf__class_weight'],
        kernel=best_model['param_clf__kernel'],
        gamma=best_model['param_clf__gamma']
    )),    
])

clf.fit(df.loc[train,'content'],df.loc[train,'labels'])

KeyboardInterrupt: 

In [27]:
df.loc[train,'content']

0       Increased social and environmental vulnerabili...
1       There exists guarded. agreement that climate c...
2       This issue of Agricultural Economics is a spec...
3       Emerging ridesharing travel could be an effect...
4       First generation biofuels are widely available...
                              ...                        
2504    This study aims to analyze greenhouse gas emis...
2505    The paper describes the state-of-the-art of ex...
2507    Deforestation of tropical forest is serious pr...
2508    Quantitative simulations of the global-scale b...
2511    A detailed description of the European Commiss...
Name: content, Length: 2282, dtype: object

In [32]:
best_model['param_clf__class_weight']

ValueError: malformed node or string: <_ast.Name object at 0x7efcb11c3e10>